# Update wiag id (in table persons) in Digitales Personenregister

<!-- 
This jupyter notebook is the first part of 3 notebooks. It needs to be run before "Datenaustausch WIAG/FactGrid, Teil 1".
Before running the code below, import data from personendatenbank using the link in wiag-vocab.adw-goe.de that says "Domherren aus dem Digitalen Personenregister".
 -->

## Requirements:
* You need sql database access to the following:
  * wiag database
  * personen register database
* CSV Export should using the "Schnell" Option. (ie, without header and with commas as the delimiter)

Run the line below (without the # symbol) to install pandas if not already installed

In [1]:
#!py -m pip install pandas

In [2]:
import requests
import csv
import os
import pandas as pd
import json
import re
import os
from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d')

Change the path below to where you store your csv files

In [5]:
input_path = r"C:\Users\khan32\Documents\WIAGweb2\notebooks\input_files"

# Get data from wiag

Run the following sql query on the wiag database (https://vwebfile.gwdg.de/phpmyadmin/) and then export the results as csv. Please refer to the instructions at [Run_SQL_Query_and_Export_CSV.md](https://github.com/WIAG-ADW-GOE/WIAGweb2/blob/main/notebooks/sync_notebooks/docs/Run_SQL_Query_and_Export_CSV.md) for a detailed explanation of running the sql command below. 


```sql
SELECT DISTINCT 
  i.id,
  CASE 
    WHEN t_id.epc IS NOT NULL THEN t_id.epc
    WHEN t_id.epc IS NULL AND t_id.can IS NOT NULL THEN t_id.can
    WHEN t_id.epc IS NULL AND t_id.can IS NULL AND t_id.dreg_can IS NOT NULL THEN t_id.dreg_can
  END AS id_public,
  uext.value AS gsn
FROM 
  item AS i
JOIN 
  url_external AS uext ON uext.item_id = i.id AND uext.authority_id = 200
JOIN 
  item_name_role AS inr ON inr.item_id_name = i.id
JOIN 
  (SELECT DISTINCT 
      ic.item_id AS item_id,
      ic_ii.id_public AS epc,
      ic_iii.id_public AS can,
      ic_iv.id_public AS dreg_can
   FROM 
      item_corpus AS ic
   LEFT JOIN 
      item_corpus AS ic_ii ON ic_ii.item_id = ic.item_id AND ic_ii.corpus_id = 'epc'
   LEFT JOIN 
      item_corpus AS ic_iii ON ic_iii.item_id = ic.item_id AND ic_iii.corpus_id = 'can'
   LEFT JOIN 
      item_corpus AS ic_iv ON ic_iv.item_id = ic.item_id AND ic_iv.corpus_id = 'dreg-can'
   WHERE 
      ic.corpus_id IN ('epc', 'can', 'dreg-can')
  ) AS t_id ON t_id.item_id = i.id
WHERE 
  i.is_online = 1;
```

Please change the filename below so that it matches the file that you just downloaded above.

In [6]:
filename = 'item_2024_12_02.csv'

In [7]:
orig_ic_df = pd.read_csv(os.path.join(input_path, filename), names=["id", "wiag_id", "pd_id"])
ic_df = orig_ic_df
ic_df

,id,wiag_id,pd_id
0,13008,WIAG-Pers-EPISCGatz-21120-001,006-00056-001
1,13009,WIAG-Pers-EPISCGatz-21119-001,070-01102-001
2,13038,WIAG-Pers-EPISCGatz-21018-001,084-00204-001
3,13051,WIAG-Pers-EPISCGatz-21004-001,006-00074-001
4,13052,WIAG-Pers-EPISCGatz-21002-001,006-00070-001
...,...,...,...
11043,16588,WIAG-Pers-EPISCGatz-10098-001,076-00804-001
11044,17054,WIAG-Pers-EPISCGatz-10715-001,082-00940-001
11045,59821,WIAG-Pers-EPISCGatz-10667-001,068-01800-001
11046,14888,WIAG-Pers-EPISCGatz-03418-001,059-00072-001


## Check data from wiag and remove identical records

### The following code block should be empty

In [8]:
ic_df[ic_df['pd_id'].isna()]

,id,wiag_id,pd_id


# Identical records in wiag

Output is the gsn id of such records. Such entries are displayed with more detailed and should be fixed manually. The end of this notebook deals with such cases.

Note: 046-02872-001 is expected to be present. (You can look at it on [wiag here](https://wiag-vocab.adw-goe.de/id/WIAG-Pers-EPISCGatz-03210-001)).

In [12]:
gp_df = ic_df.groupby('pd_id').count()
duplicate_wiag_gsns = gp_df[gp_df['id'] > 1].index.to_list()
duplicate_wiag_gsns

[]

In [13]:
ic_df = ic_df[~ic_df['pd_id'].isin(duplicate_wiag_gsns)]

In [14]:
# ic_df['pr_imported'] = ic_df.wiag_id.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)
# ic_df = ic_df.sort_values(by=['pr_imported'])
# lower_df = ic_df.groupby('pd_id')[['wiag_id', 'pd_id', 'pr_imported']].head()
# lower_df

In [15]:
# ic_df.groupby('pd_id', as_index=False)['pr_imported'].min()

In [16]:
# selection = ic_df.groupby('pd_id')['pr_imported'].min()
# selection = selection.index.tolist()
# lower_df = ic_df[ic_df.pd_id.isin(selection)]
# lower_df

In [17]:
# ic_df[ic_df.wiag_id.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]

# Get data from Personenregister

Run the following sql query on the Personenregister database and then export the results as csv. The exported csv needs to be in the same directory.

```sql
SELECT persons.wiag, persons.id, gsn.id, gsn.nummer 
FROM items 
INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online" 
INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0 
WHERE persons.wiag IS NOT NULL AND persons.wiag != '' 
group by persons.wiag 
having gsn.id=min(gsn.id)
```

<!--
old version
```sql
SELECT persons.wiag, MIN(gsn.nummer)
FROM items
INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online"
INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0
WHERE persons.wiag IS NOT NULL AND persons.wiag != ''
GROUP BY persons.wiag
``` -->


### (Ignore) Another useful query 

```sql
select DISTINCT persons.wiag, gsn.id, gsn.nummer
from persons
inner join gsn ON gsn.item_id = persons.item_id
where persons.wiag in (
    SELECT persons.wiag
    FROM items
    INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online"
    INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0
    WHERE persons.wiag IS NOT NULL AND persons.wiag != ''
    GROUP BY persons.wiag
    having count(*) > 1
)
order by persons.wiag
```

The filename below can be changed.

In [20]:
filename = 'pr_2024-12-02.csv'

In [21]:
pr_df = pd.read_csv(os.path.join(input_path, filename), names=["wiag_id", "id", "gsn_table_id", "pd_id"])
pr_df

,wiag_id,id,gsn_table_id,pd_id
0,Q112009,337342,396166,086-00454-001
1,Q55122442,339060,402484,055-01193-001
2,WIAG-Pers-CANON-10014-001,226042,192200,054-02832-001
3,WIAG-Pers-CANON-10032-001,339026,402386,032-02373-001
4,WIAG-Pers-CANON-10047-001,339024,402382,029-03106-001
...,...,...,...,...
10997,WIAG-Pers-EPISCGatz-21735-001,252575,261488,061-06324-001
10998,WIAG-Pers-EPISCGatz-21750-001,219379,189218,052-02662-001
10999,WIAG-Pers-EPISCGatz-21751-001,303871,328143,077-00515-001
11000,WIAG-Pers-EPISCGatz-21766-001,297929,314418,073-01566-001


## Check if there are any problems with the data

If this is empty, there are no problems.

In [23]:
pr_df[pr_df['pd_id'].isna()]

,wiag_id,id,gsn_table_id,pd_id


In [24]:
pr_df_gp = pr_df.groupby('wiag_id').count()
pr_df_gp[pr_df_gp['pd_id'] > 1]

,id,gsn_table_id,pd_id
wiag_id,,,


# Compare records

In [25]:
joined_df = ic_df.merge(pr_df, on='pd_id', suffixes=('_wiag', '_pd'))
joined_df

,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
0,13008,WIAG-Pers-EPISCGatz-21120-001,006-00056-001,WIAG-Pers-EPISCGatz-21120-001,334162,390898
1,13009,WIAG-Pers-EPISCGatz-21119-001,070-01102-001,WIAG-Pers-EPISCGatz-21119-001,321113,363804
2,13038,WIAG-Pers-EPISCGatz-21018-001,084-00204-001,WIAG-Pers-EPISCGatz-21018-001,331642,382803
3,13051,WIAG-Pers-EPISCGatz-21004-001,006-00074-001,WIAG-Pers-EPISCGatz-21004-001,300667,323238
4,13052,WIAG-Pers-EPISCGatz-21002-001,006-00070-001,WIAG-Pers-EPISCGatz-21002-001,307523,338553
...,...,...,...,...,...,...
10990,16588,WIAG-Pers-EPISCGatz-10098-001,076-00804-001,WIAG-Pers-EPISCGatz-10098-001,324766,371909
10991,17054,WIAG-Pers-EPISCGatz-10715-001,082-00940-001,WIAG-Pers-EPISCGatz-10715-001,325009,370938
10992,59821,WIAG-Pers-EPISCGatz-10667-001,068-01800-001,WIAG-Pers-EPISCGatz-10667-001,338808,401286
10993,14888,WIAG-Pers-EPISCGatz-03418-001,059-00072-001,WIAG-Pers-EPISCGatz-03418-001,335581,394153


In [26]:
unequal_df = joined_df[joined_df['wiag_id_wiag'] != joined_df['wiag_id_pd']]
print(len(unequal_df))
unequal_df

39


,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
10564,62200,WIAG-Pers-CANON-53445-001,052-02090-001,WIAG-Pers-CANON-83871-001,218807,188646
10585,40165,WIAG-Pers-CANON-25701-001,021-00603-001,WIAG-Pers-CANON-81202-001,75990,63028
10631,59903,WIAG-Pers-CANON-45751-001,007-02183-001,WIAG-Pers-CANON-81011-001,24005,11132
10637,60774,WIAG-Pers-CANON-51356-001,054-01166-001,WIAG-Pers-CANON-83974-001,224376,131866
10639,62342,WIAG-Pers-CANON-49407-001,084-00687-001,WIAG-Pers-CANON-91097-001,331721,383286
10652,40215,WIAG-Pers-CANON-25779-001,054-01582-001,WIAG-Pers-CANON-84016-001,224792,190951
10666,62370,WIAG-Pers-CANON-53414-001,002-02448-001,WIAG-Pers-CANON-88020-001,320592,362807
10688,49117,WIAG-Pers-CANON-45036-001,068-01830-001,WIAG-Pers-CANON-84557-001,274841,205391
10698,36718,WIAG-Pers-CANON-17971-001,076-01492-001,WIAG-Pers-CANON-85563-001,299073,316032
10704,62464,WIAG-Pers-CANON-53546-001,052-02292-001,WIAG-Pers-CANON-85631-001,300675,323258


In [27]:
# check_list = list(map(lambda x: x[-9:-8] == '8' or x[-9:-8] == '9', unequal_df[unequal_df['wiag_id_x'].isna()]['wiag_id_y'].to_list()))
# any(check_list)
# empty_wiag = unequal_df[unequal_df['wiag_id_wiag'].isna()]
# diff_df = empty_wiag[empty_wiag.wiag_id_pd.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]
# diff_df

In [28]:
# unequal_df[unequal_df['wiag_id_wiag'].isna()]

In [29]:
# unequal_df = unequal_df[~unequal_df['wiag_id_wiag'].isna()]
# unequal_df

In [30]:
# links_df = unequal_df.copy()
# links_df['pd_link'] = links_df.apply(lambda row: "http://personendatenbank.germania-sacra.de/index/gsn/" + row["pd_id"], axis = 1)
# links_df
# links_df['current_wiag_link'] = links_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_wiag"], axis = 1)
# links_df
# links_df['pd_wiag_link'] = links_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_pd"], axis = 1)
# links_df.head()

In [31]:
# links_df[['pd_link', 'current_wiag_link', 'pd_wiag_link']].to_csv(f'inconsistent_data_links_{today_string}.csv', sep=';')

## Generate sql file to run on Personenregister

In [32]:
query = "LOCK TABLES persons WRITE;\n"
for _, row in unequal_df.iterrows():
    query += f"""
    UPDATE persons
    SET wiag = '{row['wiag_id_wiag']}'
    WHERE id = {row['id_pd']}; -- id: {row['pd_id']}
"""
query += "\nUNLOCK TABLES;"
with open(f'update_personen_register_{today_string}.sql', 'w') as file:
    file.write(query)

TODO: Mention 

# Work on people with more than one entry in wiag with the same gsn id

In [25]:
dupl_ppl = orig_ic_df[orig_ic_df['pd_id'].isin(duplicate_wiag_gsns)]
dupl_ppl

,id,wiag_id,pd_id
1319,17819,WIAG-Pers-EPISCGatz-21305-001,046-02872-001
9645,14702,WIAG-Pers-EPISCGatz-03210-001,046-02872-001


In [40]:
dupl_join_df = dupl_ppl.merge(pr_df, on='pd_id', suffixes=('_wiag', '_pd'))
dupl_join_df

,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
0,17819,WIAG-Pers-EPISCGatz-21305-001,046-02872-001,WIAG-Pers-EPISCGatz-03210-001,334165,390910
1,14702,WIAG-Pers-EPISCGatz-03210-001,046-02872-001,WIAG-Pers-EPISCGatz-03210-001,334165,390910
2,59583,WIAG-Pers-CANON-25464-001,082-00384-001,WIAG-Pers-CANON-25464-001,324601,370382
3,60772,WIAG-Pers-CANON-25465-001,082-00384-001,WIAG-Pers-CANON-25464-001,324601,370382


## Manually fix the following entries on personendatenbank if the code below produces any output

In [41]:
for gsn in duplicate_wiag_gsns:
    pd_wiag_id = pr_df[pr_df['pd_id'] == gsn]['wiag_id'].values[0]
    
    if len(dupl_ppl[dupl_ppl['wiag_id'] == pd_wiag_id]) == 0:
        # latest wiag id absent in pr
        print(dupl_ppl[dupl_ppl['pd_id'] == gsn])

In [57]:
### https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CANON-12751-001 doesn't exist on wiag???
### check for fertig status

In [58]:
# SELECT *  FROM url_external WHERE authority_id=200

In [59]:
# select id_in_source from item 
# where id_in_source is not NULL 
# and id_in_source!=''
# and id in 
# (SELECT item_id FROM url_external WHERE authority_id=200)

In [60]:
# select * from person where id in
# (select id_in_source from item 
# where id_in_source is not NULL 
# and id_in_source!=''
# and id in 
# (SELECT item_id FROM url_external WHERE authority_id=200))

In [61]:
# SELECT * FROM `item_corpus` 
# where item_id in  (SELECT item_id FROM url_external WHERE authority_id=200)

In [62]:
# SELECT i.id_public, i.item_id, u.value 
# FROM `item_corpus` i 
# INNER JOIN (
#     select item_id, value FROM url_external WHERE authority_id=200
# ) u on i.item_id = u.item_id

In [63]:
# example complex query
# http://personendatenbank.germania-sacra.de/api/v1.0/person?
# query[0][field]=person.vorname&
# query[0][value]=b*&
# query[0][operator]=like&
# query[0][connector]=or&
# query[1][field]=person.familienname&
# query[1][value]=b*&
# query[1][operator]=like&
# format=json-ld


In [29]:
joined_df[joined_df['wiag_id_wiag'] == 'WIAG-Pers-CANON-52631-001']

,id_wiag,wiag_id_wiag,pd_id,wiag_id_pd,id_pd,gsn_table_id
10909,57249,WIAG-Pers-CANON-52631-001,083-03531-001,WIAG-Pers-CANON-52631-001,326619,378197
